In [1]:
from utils import load_env_vars
from openai import OpenAI
import time, json, os, tqdm

load_env_vars()

# Initialize OpenAI client
client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])

In [2]:
from utils_gpt import prepare_training_data

task = "R"

train_input_fn = f"data/finetune_{task}_train.json"
train_output_fn = f"data/finetune_{task}_train_gpt_format.jsonl"

prepare_training_data(train_input_fn, train_output_fn)

dict_keys(['original_id', 'split', 'source', 'type', 'sample_type', 'paragraph1', 'paragraph2', 'reference_preference', 'text_input', 'output', 'id'])


'data/finetune_P_train_gpt_format.jsonl'

In [3]:
training_file = client.files.create(file=open(train_output_fn, 'rb'), purpose='fine-tune')

training_file

FileObject(id='file-dhox0F9EpP5FBS2GDQOGN83C', bytes=4892222, created_at=1731273320, filename='finetune_P_train_gpt_format.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [ ]:
base_model = "gpt-4o-mini-2024-07-18"
suffix = f"lamp-4o-mini-{task}"

job = client.fine_tuning.jobs.create(training_file=training_file.id, model=base_model, suffix=suffix)

In [12]:
from datetime import datetime

while True:
    job_status = client.fine_tuning.jobs.retrieve(job.id)
    if job_status.status == "running":
        estimated_completion = datetime.fromtimestamp(job_status.estimated_finish)
        print(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] Status: {job_status.status}, Estimated completion: {estimated_completion}")
    else:
        print(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] Status: {job_status.status}")
    
    if job_status.status in ['succeeded', 'failed']:
        if job_status.status == "succeeded":
            print(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] Fine-tuning job completed successfully")     
            print(f"Model: {job_status.fine_tuned_model}")       
        else:
            print(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] Fine-tuning job failed")
        break
        
    time.sleep(60)  # Check status every minute



[2024-11-10 16:55:19] Status: succeeded
[2024-11-10 16:55:19] Fine-tuning job completed successfully
Model: ft:gpt-4o-mini-2024-07-18:tobias-schnabel::ASAA2MGu


In [11]:
job_status

FineTuningJob(id='ftjob-aSqwH2bGPemsSmMuwxNNy2s0', created_at=1731273445, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-mini-2024-07-18:tobias-schnabel::ASAA2MGu', finished_at=1731275605, hyperparameters=Hyperparameters(n_epochs=3, batch_size=3, learning_rate_multiplier=1.8), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-BIvrpwd05MNwVEsV20uuOQvB', result_files=['file-M3HPwnyUSu43t36DaPMpjqY9'], seed=683719816, status='succeeded', trained_tokens=3009744, training_file='file-dhox0F9EpP5FBS2GDQOGN83C', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)